In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import os



# Створюємо порожній список для збереження всіх даних
data = []

# Перебираємо активності
activities = ["idle", "running", "stairs", "walking"]
for activity in activities:
    activity_folder = os.path.join("data", activity)
    
    # Перебираємо файли в папці активності
    for filename in os.listdir(activity_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(activity_folder, filename)
            
            # Завантажуємо файл
            df = pd.read_csv(file_path)
            
            # Розраховуємо часові ознаки
            time_features = df[['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']]
            time_features = time_features.agg(['mean', 'std', 'median', 'min', 'max']).T
            time_features['activity'] = activity
            data.append(time_features)

# Об'єднуємо всі дані в один DataFrame
data_df = pd.concat(data, ignore_index=True)

# Розділення даних на навчальний і тестовий набори
X = data_df.drop('activity', axis=1)
y = data_df['activity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизація ознак
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Навчання моделей
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_scaled, y_train)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Оцінка моделей
svm_pred = svm_model.predict(X_test_scaled)
rf_pred = rf_model.predict(X_test_scaled)

print("SVM Results:")
print("Accuracy:", accuracy_score(y_test, svm_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, svm_pred))
if set(y_test) == set(svm_pred):
    print("Classification Report:\n", classification_report(y_test, svm_pred))
else:
    print("Some classes have no predictions in SVM.")

print("\nRandom Forest Results:")
print("Accuracy:", accuracy_score(y_test, rf_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, rf_pred))
if set(y_test) == set(rf_pred):
    print("Classification Report:\n", classification_report(y_test, rf_pred))
else:
    print("Some classes have no predictions in Random Forest.")



SVM Results:
Accuracy: 0.9025270758122743
Confusion Matrix:
 [[ 612    0    0    0]
 [   0 1912    0  154]
 [  12    9    0   73]
 [   0  130    0  976]]
Some classes have no predictions in SVM.

Random Forest Results:
Accuracy: 0.9886539453326457
Confusion Matrix:
 [[ 612    0    0    0]
 [   0 2057    0    9]
 [   1    4   78   11]
 [   0   19    0 1087]]
Classification Report:
               precision    recall  f1-score   support

        idle       1.00      1.00      1.00       612
     running       0.99      1.00      0.99      2066
      stairs       1.00      0.83      0.91        94
     walking       0.98      0.98      0.98      1106

    accuracy                           0.99      3878
   macro avg       0.99      0.95      0.97      3878
weighted avg       0.99      0.99      0.99      3878
